<a href="https://colab.research.google.com/github/miroorc/Capstone-Project/blob/main/BART__without_summary.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install --upgrade datasets

In [ ]:
pip install transformers

In [ ]:
pip install transformers[torch]

In [ ]:
pip install scikit-learn

In [ ]:
#load the ecthra dataset from lex_glue
from datasets import load_dataset
ecthra = load_dataset("lex_glue", "ecthr_a")

In [ ]:
#ecthra['train']=ecthra['train'].select([10])
#ecthra['test']=ecthra['test'].select(range(10))
#ecthra['validation']=ecthra['validation'].select(range(10))

In [ ]:
#Extract the train, test, and validation parts of the ecthra dataset.
train=ecthra['train']
validation=ecthra['validation']
test=ecthra['test']

In [ ]:
#convert label to one-hot form
import numpy as np
num_classes = 10

def one_hot_encode_multi(labels, num_classes):
    encoded_labels = np.zeros((len(labels),num_classes))
    for i, sublist in enumerate(labels):
        encoded_labels[i, sublist] = 1
    return encoded_labels

In [ ]:
train_one_hot=one_hot_encode_multi(train['labels'],10)
train_one_hot = train_one_hot.tolist()
train=train.add_column(name="labels_one_hot", column=train_one_hot)

In [ ]:
test_one_hot=one_hot_encode_multi(test['labels'],10)
test_one_hot = test_one_hot.tolist()
test=test.add_column(name="labels_one_hot", column=test_one_hot)

In [ ]:
validation_one_hot=one_hot_encode_multi(validation['labels'],10)
validation_one_hot = validation_one_hot.tolist()
validation=validation.add_column(name="labels_one_hot", column=validation_one_hot)

In [ ]:
#clean the dataset to remove meaningless characters
import re
def format_text(text):
    text_format = []
    text=[[re.sub(r"^\d+\.\s*","",s) for s in sublst] for sublst in text]
    for each in text:
        text_format.append(''.join(each))
    return text_format

In [ ]:
train_text_format=format_text(train['text'])
train=train.remove_columns('text')
train=train.add_column(name="text", column=train_text_format)
ecthra['train']=train

In [ ]:
test_text_format=format_text(test['text'])
test=test.remove_columns('text')
test=test.add_column(name="text", column=test_text_format)
ecthra['test']=test

In [ ]:
validation_text_format=format_text(validation['text'])
validation=validation.remove_columns('text')
validation=validation.add_column(name="text", column=validation_text_format)
ecthra['validation']=validation

In [ ]:
ecthra=ecthra.remove_columns('labels')
ecthra=ecthra.rename_column('labels_one_hot', 'labels')

In [ ]:
from transformers import BartTokenizerFast, BartForSequenceClassification

In [ ]:
#add fine-tuning model


model_ckpt = "facebook/bart-base"
tokenizer = BartTokenizerFast.from_pretrained(model_ckpt, problem_type="multi_label_classification")



In [ ]:
def tokenize_and_encode(examples):
  return tokenizer(examples["text"], truncation=True)

In [ ]:
cols = ecthra["train"].column_names
cols.remove("labels")
ds_enc = ecthra.map(tokenize_and_encode, batched=True, remove_columns=cols)


Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

In [ ]:
ds_enc['train']

Dataset({
    features: ['labels', 'input_ids', 'attention_mask'],
    num_rows: 9000
})

In [ ]:
len(ds_enc['train'][0]['input_ids'])

1024

In [ ]:
ds_enc.set_format("torch")

In [ ]:
num_labels=10
model = BartForSequenceClassification.from_pretrained(model_ckpt, num_labels=num_labels, problem_type="multi_label_classification",ignore_mismatched_sizes=True)

Some weights of BartForSequenceClassification were not initialized from the model checkpoint at facebook/bart-base and are newly initialized: ['classification_head.out_proj.bias', 'classification_head.out_proj.weight', 'classification_head.dense.bias', 'classification_head.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
from transformers import (AutoTokenizer, AutoModelForSequenceClassification,
                          TrainingArguments, Trainer)

In [ ]:
model.to('cuda')

BartForSequenceClassification(
  (model): BartModel(
    (shared): Embedding(50265, 768, padding_idx=1)
    (encoder): BartEncoder(
      (embed_tokens): Embedding(50265, 768, padding_idx=1)
      (embed_positions): BartLearnedPositionalEmbedding(1026, 768)
      (layers): ModuleList(
        (0-5): 6 x BartEncoderLayer(
          (self_attn): BartAttention(
            (k_proj): Linear(in_features=768, out_features=768, bias=True)
            (v_proj): Linear(in_features=768, out_features=768, bias=True)
            (q_proj): Linear(in_features=768, out_features=768, bias=True)
            (out_proj): Linear(in_features=768, out_features=768, bias=True)
          )
          (self_attn_layer_norm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
          (activation_fn): GELUActivation()
          (fc1): Linear(in_features=768, out_features=3072, bias=True)
          (fc2): Linear(in_features=3072, out_features=768, bias=True)
          (final_layer_norm): LayerNorm((768,), eps

In [ ]:
# define the arguments
batch_size = 4
metric_name = "f1"

In [ ]:
args = TrainingArguments(
    f"output",
    evaluation_strategy = "epoch",
    save_strategy = "epoch",
    learning_rate=5e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=8,
    weight_decay=0.05,
    load_best_model_at_end=True,
    metric_for_best_model=metric_name,
)

In [ ]:
# reference: https://jesusleal.io/2021/04/21/Longformer-multilabel-classification/
from sklearn.metrics import f1_score, roc_auc_score, accuracy_score,recall_score
from transformers import EvalPrediction
import sklearn.metrics as skm
import torch
import json


def multi_label_metrics(predictions, labels, threshold=0.6):
    # first, apply sigmoid on predictions which are of shape (batch_size, num_labels)
    sigmoid = torch.nn.Sigmoid()
    probs = sigmoid(torch.Tensor(predictions))
    # next, use threshold to turn them into integer predictions
    y_pred = np.zeros(probs.shape)
    y_pred[np.where(probs >= threshold)] = 1
    # finally, compute metrics
    y_true = labels
    recall=recall_score(y_true, y_pred, average='macro')
    f1_micro_average = f1_score(y_true=y_true, y_pred=y_pred, average='micro')
    roc_auc = roc_auc_score(y_true, y_pred, average = 'micro')
    accuracy = accuracy_score(y_true, y_pred)
    # return as dictionary
    #matrixes = skm.confusion_matrix(y_true, y_pred)
    cm = skm.multilabel_confusion_matrix(y_true, y_pred).tolist()
    metrics = {'f1': f1_micro_average,
               'roc_auc': roc_auc,
               'accuracy': accuracy,
               'recall':recall,
               'matrixes':cm

               }
    return metrics

def compute_metrics(p: EvalPrediction):
    preds = p.predictions[0] if isinstance(p.predictions,
            tuple) else p.predictions
    result = multi_label_metrics(
        predictions=preds,
        labels=p.label_ids)
    return result

In [ ]:
#define trainer
trainer = Trainer(
    model=model, args=args,
    train_dataset=ds_enc["train"],
    eval_dataset=ds_enc["validation"],
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)


In [ ]:
trainer.evaluate()

You're using a BartTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Trainer is attempting to log a value of "[[[943, 0], [57, 0]], [[800, 7], [190, 3]], [[796, 17], [185, 2]], [[698, 2], [300, 0]], [[698, 215], [80, 7]], [[909, 87], [4, 0]], [[895, 63], [41, 1]], [[956, 11], [33, 0]], [[982, 0], [18, 0]], [[821, 40], [137, 2]]]" of type <class 'list'> for key "eval/matrixes" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.


{'eval_loss': 0.6738670468330383,
 'eval_f1': 0.019775873434410018,
 'eval_roc_auc': 0.4823551137562787,
 'eval_accuracy': 0.133,
 'eval_recall': 0.014489701174965203,
 'eval_matrixes': [[[943, 0], [57, 0]],
  [[800, 7], [190, 3]],
  [[796, 17], [185, 2]],
  [[698, 2], [300, 0]],
  [[698, 215], [80, 7]],
  [[909, 87], [4, 0]],
  [[895, 63], [41, 1]],
  [[956, 11], [33, 0]],
  [[982, 0], [18, 0]],
  [[821, 40], [137, 2]]],
 'eval_runtime': 104.6411,
 'eval_samples_per_second': 9.556,
 'eval_steps_per_second': 2.389}

In [ ]:
import torch
torch.cuda.empty_cache()
trainer.train()

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch,Training Loss,Validation Loss,F1,Roc Auc,Accuracy,Recall,Matrixes
1,0.159400,0.168626,0.598515,0.737830,0.473000,0.365792,"[[[934, 9], [30, 27]], [[796, 11], [116, 77]], [[766, 47], [62, 125]], [[625, 75], [110, 190]], [[913, 0], [87, 0]], [[995, 1], [3, 1]], [[958, 0], [42, 0]], [[963, 4], [12, 21]], [[982, 0], [18, 0]], [[841, 20], [56, 83]]]"
2,0.134200,0.156700,0.663152,0.785065,0.528000,0.516308,"[[[936, 7], [21, 36]], [[774, 33], [61, 132]], [[771, 42], [51, 136]], [[670, 30], [148, 152]], [[871, 42], [45, 42]], [[993, 3], [2, 2]], [[940, 18], [15, 27]], [[965, 2], [22, 11]], [[982, 0], [18, 0]], [[830, 31], [48, 91]]]"
3,0.109300,0.164898,0.686819,0.812066,0.536000,0.573154,"[[[928, 15], [18, 39]], [[785, 22], [77, 116]], [[794, 19], [72, 115]], [[628, 72], [107, 193]], [[855, 58], [38, 49]], [[995, 1], [3, 1]], [[935, 23], [9, 33]], [[962, 5], [10, 23]], [[978, 4], [18, 0]], [[815, 46], [15, 124]]]"
4,0.088400,0.172298,0.696652,0.815127,0.544000,0.543143,"[[[926, 17], [17, 40]], [[783, 24], [64, 129]], [[780, 33], [55, 132]], [[607, 93], [85, 215]], [[868, 45], [37, 50]], [[996, 0], [4, 0]], [[945, 13], [18, 24]], [[966, 1], [8, 25]], [[980, 2], [15, 3]], [[845, 16], [60, 79]]]"
5,0.079900,0.175672,0.686838,0.807692,0.534000,0.544459,"[[[923, 20], [20, 37]], [[785, 22], [66, 127]], [[789, 24], [68, 119]], [[636, 64], [114, 186]], [[874, 39], [55, 32]], [[996, 0], [4, 0]], [[948, 10], [18, 24]], [[965, 2], [8, 25]], [[975, 7], [13, 5]], [[807, 54], [13, 126]]]"
6,0.059200,0.191743,0.702651,0.827872,0.533000,0.598288,"[[[924, 19], [18, 39]], [[750, 57], [42, 151]], [[779, 34], [55, 132]], [[625, 75], [106, 194]], [[868, 45], [43, 44]], [[995, 1], [4, 0]], [[940, 18], [13, 29]], [[964, 3], [4, 29]], [[978, 4], [12, 6]], [[831, 30], [34, 105]]]"
7,0.047800,0.219174,0.710706,0.848070,0.529000,0.638439,"[[[915, 28], [13, 44]], [[749, 58], [40, 153]], [[776, 37], [54, 133]], [[589, 111], [79, 221]], [[868, 45], [38, 49]], [[995, 1], [4, 0]], [[941, 17], [13, 29]], [[960, 7], [3, 30]], [[974, 8], [11, 7]], [[818, 43], [25, 114]]]"
8,0.034600,0.226985,0.695734,0.831488,0.521000,0.584099,"[[[921, 22], [19, 38]], [[753, 54], [42, 151]], [[779, 34], [65, 122]], [[595, 105], [82, 218]], [[863, 50], [44, 43]], [[995, 1], [4, 0]], [[944, 14], [16, 26]], [[963, 4], [4, 29]], [[975, 7], [14, 4]], [[821, 40], [28, 111]]]"


Trainer is attempting to log a value of "[[[934, 9], [30, 27]], [[796, 11], [116, 77]], [[766, 47], [62, 125]], [[625, 75], [110, 190]], [[913, 0], [87, 0]], [[995, 1], [3, 1]], [[958, 0], [42, 0]], [[963, 4], [12, 21]], [[982, 0], [18, 0]], [[841, 20], [56, 83]]]" of type <class 'list'> for key "eval/matrixes" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
Trainer is attempting to log a value of "[[[936, 7], [21, 36]], [[774, 33], [61, 132]], [[771, 42], [51, 136]], [[670, 30], [148, 152]], [[871, 42], [45, 42]], [[993, 3], [2, 2]], [[940, 18], [15, 27]], [[965, 2], [22, 11]], [[982, 0], [18, 0]], [[830, 31], [48, 91]]]" of type <class 'list'> for key "eval/matrixes" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
Trainer is attempting to log a value of "[[[928, 15], [18, 39]], [[785, 22], [77, 116]], [[794, 19], [72, 115]], [[628, 72], [107, 193]], [[855, 58], [3

TrainOutput(global_step=18000, training_loss=0.09182254399193658, metrics={'train_runtime': 23102.7522, 'train_samples_per_second': 3.117, 'train_steps_per_second': 0.779, 'total_flos': 4.384803407785306e+16, 'train_loss': 0.09182254399193658, 'epoch': 8.0})

In [ ]:
x=trainer.evaluate()

In [ ]:
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

confusion_matrices = x['eval_matrixes']

# make confusion matrix
labels = ['Positive', 'Negative']

fig, axs = plt.subplots(2, 5, figsize=(15, 6))
for i, ax in enumerate(axs.flatten()):
    sns.heatmap(confusion_matrices[i], annot=True, fmt='d', cmap='Blues', xticklabels=labels, yticklabels=labels, ax=ax)
    ax.set_title(f'Confusion Matrix {i+1}')
    ax.set_xlabel('Predicted Label')
    ax.set_ylabel('Actual Label')

plt.tight_layout()

plt.show()

